In [20]:
#| include: false
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
cd(@__DIR__)

  Activating project at `~/gitrepos/kdheepak.github.io/blog/effect-of-type-inference-on-performance-in-julia`


In Julia, to ensure that the code you write executes fast and efficiently, it is important to benchmark frequently. There's lots of really useful tips in the [Performance Tips] section in the official documentation.

[Performance Tips]: https://docs.julialang.org/en/v1/manual/performance-tips/

In this blog post, I want to touch on one specific performance tip: containers with abstract types and type inference.

# Toy problem

Let's define a toy problem to work with.

In [21]:
abstract type Shape end
area(::Shape) = 0.0

@kwdef struct Square <: Shape
    side::Float64 = rand()
end
area(s::Square) = s.side * s.side
    
@kwdef struct Rectangle <: Shape
    width::Float64 = rand()
    height::Float64 = rand()
end
area(r::Rectangle) = r.width * r.height
    
@kwdef struct Triangle <: Shape
    base::Float64 = rand()
    height::Float64 = rand()
end
area(t::Triangle) = 1.0/2.0 * t.base * t.height

@kwdef struct Circle <: Shape
    radius::Float64 = rand()
end
area(c::Circle) = π * c.radius^2

nothing #| hide_line

We can use the builtin `Test` module to check that the code we wrote is correct.

In [22]:
using Test
@testset "Areas" begin
    @test area(Square(2)) == 4
    @test area(Rectangle(2,3)) == 6
    @test area(Triangle(2,3)) == 3
    @test area(Circle(2)) ≈ 4π
end
nothing #| hide_line

Test Summary: | Pass  Total  Time
Areas         |    4      4  0.0s


Let's also build 1 million random shapes.

In [23]:
using Random
Random.seed!(42)

COUNT = 1_000_000
shapes = [rand((Square,Rectangle,Triangle,Circle))() for _ in 1:COUNT]

nothing #| hide_line

In [24]:
#| code-fold: true
using Format
using Markdown
len = cfmt("%'d", length(shapes))
number_of(T) = cfmt("%'d", count(s->isa(s, T), shapes))

display(Markdown.md"The total number of shapes we have in the array is $len.")

display(Markdown.md"In the array, there's $(number_of(Square)) squares, $(number_of(Rectangle)) rectangles, 
$(number_of(Triangle)) triangles, and $(number_of(Circle)) circles.")

The total number of shapes we have in the array is 1,000,000.


In the array, there's 249,740 squares, 249,980 rectangles,  249,831 triangles, and 250,449 circles.


# Type inference

We can use the `typeof` function to see what the type of the data in the `shapes` variable is:

In [25]:
typeof(shapes)

Vector{Shape} (alias for Array{Shape, 1})

By default, Julia will infer the type as close to the bottom of the tree that fits all the data in the container.

For example, if we just built a vector with the same elements (e.g. `Square`), Julia will infer the container to be `Vector{Square}`.

In [26]:
typeof([Square() for _ in 1:COUNT])

Vector{Square} (alias for Array{Square, 1})

Here's the type trees for the shapes:

In [27]:
#| code-fold: true
for s in subtypes(Shape)
    println(join(string.(supertypes(s)), " <: "))
end

Circle <: Shape <: Any
Rectangle <: Shape <: Any
Square <: Shape <: Any
Triangle <: Shape <: Any


Let's define a function that calculates the `area` for all the shapes and adds them all up

In [36]:
main1(shapes) = sum(area, shapes);

Let's filter out shapes of a specific kind so that each array contains data of the same type. 
You might think to write this function like so:

In [35]:
bad_shapes_by_type(::Type{T}, shapes) where T = filter(s -> isa(s, T), shapes);

Julia is a dynamic language. And it can be easy to accidentally construct a container with an abstract type for the type parameter of a generic type.

In [31]:
shape_arr1 = bad_shapes_by_type(Square, shapes)
shape_arr2 = bad_shapes_by_type(Rectangle, shapes)
shape_arr3 = bad_shapes_by_type(Triangle, shapes)
shape_arr4 = bad_shapes_by_type(Circle, shapes)

@show typeof(shape_arr1)
@show typeof(shape_arr2)
@show typeof(shape_arr3)
@show typeof(shape_arr4)

nothing #| hide_line

typeof(shape_arr1) = Vector{Shape}
typeof(shape_arr2) = Vector{Shape}
typeof(shape_arr3) = Vector{Shape}
typeof(shape_arr4) = Vector{Shape}


This can happen if the Julia compiler cannot infer the types at the "compile time" of the function.

Here, `Shape` is an abstract type, even if `Vector{Shape}` is concrete.

In [32]:
@show isconcretetype(Shape)
@show isconcretetype(Vector{Shape})
nothing #| hide_line

isconcretetype(Shape) = false
isconcretetype(Vector{Shape}) = true


For better performance in Julia, it helps to have concrete types in the generic parameters for a container. 
We can do this by helping the compiler figure out the correct concrete type parameter by explicitly listing it before the brackets for constructing the array, i.e. `T[...]`.

In [34]:
good_shapes_by_type(::Type{T}, shapes) where T = T[shape for shape in filter(s -> isa(s, T), shapes)];

In [37]:
square_arr = good_shapes_by_type(Square, shapes)
rectangle_arr = good_shapes_by_type(Rectangle, shapes)
triangle_arr = good_shapes_by_type(Triangle, shapes)
circle_arr = good_shapes_by_type(Circle, shapes)

@show typeof(square_arr)
@show typeof(rectangle_arr)
@show typeof(triangle_arr)
@show typeof(circle_arr)

nothing #| hide_line

typeof(square_arr) = Vector{Square}
typeof(rectangle_arr) = Vector{Rectangle}
typeof(triangle_arr) = Vector{Triangle}
typeof(circle_arr) = Vector{Circle}


# Benchmarks

Let's combine these arrays back into three different vectors that have different type parameters for illustration purposes:

In [38]:
sorted_shapes_shape = vcat(square_arr, rectangle_arr, triangle_arr, circle_arr);
sorted_shapes_any = Any[s for s in sorted_shapes_shape];
sorted_shapes_union = Union{Square, Rectangle, Triangle, Circle}[s for s in sorted_shapes_shape];

@show typeof(sorted_shapes_shape)
@show typeof(sorted_shapes_any)
@show typeof(sorted_shapes_union)

nothing #| hide_line

typeof(sorted_shapes_shape) = Vector{Shape}
typeof(sorted_shapes_any) = Vector{Any}
typeof(sorted_shapes_union) = Vector{Union{Circle, Rectangle, Square, Triangle}}


We can benchmark the performance of these different types using `BenchmarkTools`:

In [39]:
using BenchmarkTools

@show typeof(shapes)
@benchmark main1($shapes)

typeof(shapes) = Vector{Shape}


BenchmarkTools.Trial: 122 samples with 1 evaluation.
 Range (min … max):  39.079 ms … 46.225 ms  ┊ GC (min … max): 0.00% … 8.63%
 Time  (median):     40.062 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   41.271 ms ±  2.184 ms  ┊ GC (mean ± σ):  3.16% ± 3.97%

   ▃█ ▁                                        ▁               
  ▇██▆█▆█▃▆▆▃▄▃▃▄▄▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▃▃▇▃▅▄▆▅█▆▆▁▃▁▁▁▁▁▁▁▃▄ ▃
  39.1 ms         Histogram: frequency by time        45.6 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 1999999.

Let's run the benchmark for `sorted_shapes_shape` and `sorted_shapes_any`.

In [40]:
#| code-fold: true

@show typeof(sorted_shapes_shape)
@benchmark main1($sorted_shapes_shape)

typeof(sorted_shapes_shape) = Vector{Shape}


BenchmarkTools.Trial: 149 samples with 1 evaluation.
 Range (min … max):  31.097 ms … 38.119 ms  ┊ GC (min … max): 0.00% … 10.57%
 Time  (median):     32.549 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   33.608 ms ±  2.197 ms  ┊ GC (mean ± σ):  3.97% ±  4.97%

   ▂█▇  ▁ ▁▃▂                                                  
  ▆████▄█▄████▄▃▄▆▆▇▇▄▃▁▁▁▁▁▁▁▁▃▁▃▃▃▁▁▆▄▆▄▄▇▆▆█▃▆█▃▁▆▃▁▃▃▃▃▃▄ ▃
  31.1 ms         Histogram: frequency by time        38.1 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 1999999.

In [61]:
#| code-fold: true

@show typeof(sorted_shapes_any)
@benchmark main1($sorted_shapes_any)

typeof(sorted_shapes_any) = Vector{Any}


BenchmarkTools.Trial: 149 samples with 1 evaluation.
 Range (min … max):  31.384 ms … 39.205 ms  ┊ GC (min … max): 0.00% … 11.42%
 Time  (median):     32.467 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   33.643 ms ±  2.315 ms  ┊ GC (mean ± σ):  3.92% ±  5.08%

  ▅▄█▂▂▂   ▁                                                   
  ██████▅▇▇█▆▁▆▁▃▃▄▁▁▁▁▃▁▁▁▁▁▁▁▁▁▃▃▄▅▅▅▄▆▇▄▆▃▃▄▄▄▅▃▄▃▃▁▄▁▁▁▁▃ ▃
  31.4 ms         Histogram: frequency by time        38.9 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 1999999.

Both benchmarks for `Vector{Shape}` and `Vector{Any}` are similar in performance to each other. 

The Julia manual has the following to say:

> If you cannot avoid containers with abstract value types, it is sometimes better to parametrize with `Any` to avoid runtime type checking. E.g. `IdDict{Any, Any}` performs better than `IdDict{Type, Vector}`

What is interesting is that `Union{Circle, Rectangle, Square, Triangle}` can perform better than `Shape` or `Any` when used as a concrete type parameter.

You can see difference show up clearly in the performance benchmark:

In [42]:
#| code-fold: true

@show typeof(sorted_shapes_union)
@benchmark main1($sorted_shapes_union)

typeof(sorted_shapes_union) = Vector{Union{Circle, Rectangle, Square, Triangle}}


BenchmarkTools.Trial: 5262 samples with 1 evaluation.
 Range (min … max):  928.958 μs … 1.035 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     945.167 μs             ┊ GC (median):    0.00%
 Time  (mean ± σ):   947.885 μs ± 9.102 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                ▃█▁▁▂                                          
  ▁▁▁▁▁▁▁▁▁▂▂▂▃▅█████▄▃▂▂▂▁▁▁▁▁▂▂▂▃▂▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  929 μs          Histogram: frequency by time         986 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [43]:
#| echo: false
using Format
f = cfmt("%'d", 33.877 * 1e3 / 939)
Markdown.md"`Vector{Union{Circle, Rectangle, Square, Triangle}}` is faster than `Vector{Shape}` by roughly a factor of $f times."

`Vector{Union{Circle, Rectangle, Square, Triangle}}` is faster than `Vector{Shape}` by roughly a factor of 36 times.


It's possible to get even better performance by calculating the `sum`s for the individual arrays and summing them up at the end.

In [46]:
main2(arrs...) = sum(main1, arrs);

In [48]:
@benchmark main2($square_arr, $rectangle_arr, $triangle_arr, $circle_arr)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  272.125 μs … 463.500 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     280.416 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   283.815 μs ±   9.047 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▃▇█▆▄▅▅▃▂▂▁▁▁▁▁  ▁      ▁                             ▂
  ▃▃▄▄▆▇▇█████████████████████████████████▇█▇██▇▇▇▇▆▇▆▅▅▆▇▅▅▅▅▅ █
  272 μs        Histogram: log(frequency) by time        318 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

This performance is equivalent to having a uniform type (e.g. just `Square`s).

In [49]:
squares = [Square() for _ in 1:COUNT]

@benchmark main1($squares)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  235.458 μs … 346.208 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     235.709 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   238.844 μs ±   7.238 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▃▁▁ ▅▁  ▁                                                    ▁
  ███████▇███▇▆▇███▇▇▇█▇▇▇█▇▇▇▇▇▇██▇▇▇▆▆▆▆▇▇▆▇▆▆▆▇▆▅▆▅▆▅▅▅▅▅▅▄▄ █
  235 μs        Histogram: log(frequency) by time        269 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

# Conclusion

The key takeaway is that if you care about performance in Julia, you have to be mindful of types and type inference! Keeping types as concrete as possible is important because when type inference fails, it can propogate through your program. Even small changes to your code can improve performance significantly.

Many thanks to the helpful [Julia community on Discourse](https://discourse.julialang.org/t/unusual-non-deterministic-benchmark-results/113273/) for always offering insightful comments and advice.